In [54]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
from bokeh.palettes import Spectral5
from bokeh.io import output_notebook, curdoc

from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.charts import Histogram, Donut

from bokeh.models.formatters import DatetimeTickFormatter as dttf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [17]:
output_notebook() #loading bokeh for notebook output, will change to file for later dashboard
# output_file("test.html", title = 'test for output')


Loading BokehJS ...

In [3]:
df=pd.read_json('teamsdone.json')

In [4]:
#gathers the total number of SC2 Tournaments Played
df['totalsc2'] = df.groupby('PlayerId')['PlayerId'].transform('count')

In [5]:
#popped ('DOB' Column for now, as it returned improper values)
df.DOB = pd.to_datetime(df.DOB)
df.EndDate = pd.to_datetime(df.EndDate)

In [6]:
df.Prize_USD = df.Prize_USD / df.TeamPlayers
df.Prize_USD = (df.Prize_USD.astype(int))
df[df.TotalUSDPrize < df.Prize_USD]
df['date']=df['EndDate']
df.index = df['date']
df.sort_index()

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date
date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,100.0,136,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28
2010-03-07,22.0,be,EUR,Orly,1987-06-19,2010-03-07,1.362350,151,Olivier,Bellemans,...,100.0,136,WIN,1,Unaffiliated,2596.46,ZOTAC Cup Starcraft II #2 (EU),"[{'team': 'LowLandLions', 'history': [12681792...",6,2010-03-07
2010-03-14,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,100.0,137,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14
2010-03-21,20.0,gb,EUR,DeMusliM,1990-01-24,2010-03-21,1.353400,151,Benjamin,Baker,...,100.0,135,WIN,1,Unaffiliated,31612.56,ZOTAC Cup Starcraft II #4 (EU),"[{'team': 'EURONICS Gaming', 'history': [14542...",47,2010-03-21
2010-03-28,26.0,se,EUR,HayprO,1983-06-19,2010-03-28,1.342230,151,Hayder,Hussein,...,100.0,134,WIN,1,Unaffiliated,7966.80,ZOTAC Cup Starcraft II #5 (EU),"[{'team': 'Team Liquid', 'history': [127802880...",10,2010-03-28
2010-04-01,NaN,es,EUR,LucifroN,NaT,2010-04-01,1.357183,151,Pedro,Moreno Durán,...,500.0,678,WIN,1,1,62777.29,Go4SC2 March 2010 Monthly Finals,None,46,2010-04-01
2010-04-03,NaN,de,EUR,Mardow,NaT,2010-04-03,1.343287,151,Sebastian,Dutsch,...,100.0,134,WIN,1,Unaffiliated,134.33,Go4SC2 EU #3,[],1,2010-04-03
2010-04-04,24.0,de,EUR,GoOdy,1985-10-27,2010-04-04,1.350200,151,Sascha,Lupp,...,100.0,135,WIN,1,Unaffiliated,29551.33,ZOTAC Cup Starcraft II #6 (EU),"[{'team': 'ESC Gaming', 'history': [1351123200...",144,2010-04-04
2010-04-10,29.0,ua,EUR,White-Ra,1980-11-15,2010-04-10,1.343287,151,Aleksey,Krupnyk,...,100.0,134,WIN,1,mousesports,50614.31,Go4SC2 EU #5,"[{'team': 'mousesports', 'history': [126938880...",42,2010-04-10


In [7]:
dfid = df[df['PlayerId']==1058]
dfid.index = dfid['date']
dfid = dfid.sort_index()
dfid['CumulativePrizeToDate']=dfid.Prize_USD.cumsum()
dfid['PrizeToDate'] = dfid.CumulativePrizeToDate - dfid.Prize_USD
dfid['WinsToDate']=np.arange(len(dfid))
dfid['AverageWinsToDate']=np.where(dfid['WinsToDate']==0, 0, dfid.PrizeToDate/dfid.WinsToDate)

In [8]:
pid = df['PlayerId'].unique()
for i in pid:
    if i != 1058:
        dfi = df[df['PlayerId']==i]
        dfi.index = dfi['date']
        dfi = dfi.sort_index()
        dfi['CumulativePrizeToDate']=dfi.Prize_USD.cumsum()
        dfi['PrizeToDate'] = dfi.CumulativePrizeToDate - dfi.Prize_USD
        dfi['WinsToDate']=np.arange(len(dfi))
        dfi['AverageWinsToDate']=np.where(dfi['WinsToDate']==0, 0, dfi.PrizeToDate/dfi.WinsToDate)
        dfid = dfid.append(dfi)

In [9]:
dfid

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate
date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000
2010-03-14,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000
2010-08-15,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,...,mTw,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000
2010-08-22,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,...,mTw,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667
2010-10-03,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,...,mTw,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000
2010-10-17,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,...,mTw,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000
2010-10-23,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,...,mTw,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667
2010-11-10,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,...,mTw,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286
2010-11-24,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,...,mTw,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000


In [10]:
df = dfid
# # #checks for playerIds of players who had a team affiliation:
# pid = df[df['Team_On']!='Unaffiliated']['PlayerId'].unique()
# pid

In [11]:
# dfid = df[df['PlayerId']==1058]
# dfid = dfid.sort_index()
# hist = list(dfid['team_history'])
# b = dfid['date']
# l = []
# for h in range(len(hist)):
#     i=1
#     for j in range(len(hist[h])):
#         dc = hist[h][j]['history'][1]
# #         print(np.datetime64(dc,'ms'), b[h], i)
#         if(b[h] > np.datetime64(dc,'ms')):
#             i+=1
# #         print(hist[h][j])
# #         print(b[h])
# #         if b[h] < (np.datetime64((hist[h][it]['history'][1]),'ms')):
# # #             i+=1
#     l.append(i)
# dfid['num_teams'] = l
# #iterating
# for i in pid:
#     dfi = df[df['PlayerId']==i]
#     dfi = dfid.sort_index()
#     hist = list(dfi['team_history'])
#     b = dfi['date']
#     q = []
#     for h in range(len(hist)):
#         i=1
#         for j in range(len(hist[h])):
#             dc = hist[h][j]['history'][1]
#     #         print(np.datetime64(dc,'ms'), b[h], i)
#             if(b[h] > np.datetime64(dc,'ms')):
#                 i+=1
#         q.append(i)
#     dfi['num_teams'] = q
#     dfid = dfid.append(dfi)

In [12]:
dft = dfid
dft.describe()

,Age_at_win,ExchangeRate,GameId,PlayerId,Prize,Prize_USD,TeamPlayers,TotalUSDPrize,totalsc2,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate
count,14248.000000,19969.000000,19969.0,19969.000000,1.996900e+04,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000
mean,21.834924,0.900765,151.0,4338.052331,8.958006e+05,1185.711653,1.665532,94423.296888,71.627673,48506.761881,47321.050228,35.313836,1169.097463
std,2.998722,7.955835,0.0,6066.001574,4.934940e+06,4445.845763,2.157754,116138.686185,72.333421,76584.040015,75862.746578,46.599602,1869.827001
min,13.000000,0.000822,151.0,1000.000000,0.000000e+00,0.000000,1.000000,2.370000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,0.171842,151.0,1133.000000,1.000000e+02,75.000000,1.000000,10031.990000,20.000000,2192.000000,1627.000000,5.000000,227.142857
50%,22.000000,1.000000,151.0,1507.000000,4.500000e+02,250.000000,1.000000,44878.650000,55.000000,15320.000000,14228.000000,19.000000,670.166667
75%,24.000000,1.000000,151.0,3824.000000,5.000000e+03,908.000000,1.000000,134137.430000,98.000000,60613.000000,58920.000000,48.000000,1351.553846
max,34.000000,648.792620,151.0,40763.000000,1.000000e+08,200000.000000,19.000000,506645.420000,365.000000,506625.000000,503911.000000,364.000000,88487.000000


In [13]:
df = df.set_index(np.arange(len(df))) #changing the index to something nice

In [14]:
df['num_teams'] = np.where( df['team_history']=='Unaffiliated', 0, (df.team_history.str.len()))

In [15]:
df['num_teams'] = df['num_teams'].fillna(0)

In [16]:
p = Histogram(df['Age_at_win'].dropna(), title = 'Age of players that cash in tournaments')

In [79]:
show(p)

In [18]:
# p = Histogram(df['totalsc2'].unique(), title='Total Number of Cashes Per Player')

In [19]:
# show(p)

In [20]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate,num_teams
0,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000,5.0
1,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000,5.0
2,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,...,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000,5.0
3,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,...,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667,5.0
4,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000,5.0
5,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000,5.0
6,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,...,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667,5.0
7,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,...,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286,5.0
8,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,...,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000,5.0
9,24.0,ua,RUB,DIMAGA,1986-06-03,2010-11-28,0.031859,151,Dmytro,Filipchuk,...,66288.11,ASUS Autumn 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-28,6384,5747,9,638.555556,5.0


In [21]:
#AveragePricePerPlayer a lot trickier now. Will redo graph later

In [22]:
# p = Histogram(np.log(df[df['AveragePrizePerPlayer']>0]['AveragePrizePerPlayer'].unique()), title='Log Average Prize Per Player')

In [23]:
# show(p)

In [24]:
# p = Donut(df['CountryCode'], title = "Country of Origin")
# show(p)

In [25]:
df['country'] = 'Other'
top_countries = ['kr','fr','de','cn','us','ua','se','ru','pl']
for a in top_countries:
    df['country']= np.where(df['CountryCode']==a, a, df['country'])

# p = Donut(df['country'], title = "Country of Origin (top 10)")
# show(p)

In [27]:
df['currency'] = 'Other'
top_countries = ['USD', 'KRW', 'EUR', 'CNY', 'AUD']
for a in top_countries:
    df['currency']= np.where(df['CurrencyCode']==a, a, df['currency'])

# p = Donut(df['currency'], title = "Currency (Top 5)")

# show(p)

In [28]:
pd.set_option('max_columns', 50)

In [84]:
df.Age_at_win

0        23.0
1        23.0
2        24.0
3        24.0
4        24.0
5        24.0
6        24.0
7        24.0
8        24.0
9        24.0
10       24.0
11       24.0
12       24.0
13       24.0
14       24.0
15       24.0
16       24.0
17       24.0
18       24.0
19       24.0
20       24.0
21       24.0
22       24.0
23       24.0
24       24.0
25       25.0
26       25.0
27       25.0
28       25.0
29       25.0
         ... 
19939    18.0
19940    18.0
19941    18.0
19942    18.0
19943    19.0
19944    19.0
19945    19.0
19946    19.0
19947    19.0
19948    19.0
19949    19.0
19950    19.0
19951    19.0
19952    19.0
19953    19.0
19954    19.0
19955    19.0
19956    20.0
19957    20.0
19958    20.0
19959    20.0
19960    20.0
19961    20.0
19962    21.0
19963    21.0
19964    21.0
19965    22.0
19966    22.0
19967    22.0
19968    22.0
Name: Age_at_win, Length: 19969, dtype: float64

In [30]:
# p = figure()
# p.circle(x = df['EndDate'], y =df['Prize_USD'])
# p.xaxis.formatter = dttf.months

In [31]:
# show(p) #need to figure out date/Time on Bokeh

In [32]:
df['Team_On_Backup']=df['Team_On']

In [33]:
df['Team_On']=np.where(df['Team_On']==1, "Unaffiliated", df['Team_On']) #cleaning

In [57]:
p = Donut(df['Team_On'], title = 'Distribution of Winners')
show(p)

In [35]:
a = df.Team_On.value_counts().nlargest(26)
top_teams = list(a.index)
df['teams'] = 'Other'
for a in top_teams:
    df['teams']= np.where(df['Team_On']==a, a, df['teams'])

# p = Donut(df['teams'], title = "Teams (Top 25)")
# show(p)

In [36]:
df['RankText'].unique()

array(['WIN', '3rd', '4th', '1st', '5th-8th', '2nd', '3rd-4th', '9th-16th',
       '7th-8th', 'LOSE', '17th-20th', 'See Note', '13th-16th',
       '17th-24th', '33rd-64th', '17th-32nd', '5th-12th', '6th', 'Code S',
       '9th-12th', 'Round 3', '25th-32nd', 'Round 2', '25th-48th',
       '5th-6th', '5th', '9th-11th', 'Round 1', '11th-14th', '7th',
       '6th-8th', '2 Wins', '8 Wins', '9th-10th', '5 Wins', '1 Win', '-',
       '9th', '8th', '31st-60th', '7th-12th', '29th-36th', '3 Wins',
       '21st-24th', '1st-8th', '11th-12th', '13th', '15th-18th',
       '19th-26th', '27th-32nd', '10th-12th', '10th-11th', '7th-10th',
       '13th-14th', '37th-44th', '11th', '25th-28th', '10th',
       'DISQUALIFIED', '12th', '14th', '15th', '1st-4th', '4 Wins',
       '7 Wins'], dtype=object)

In [37]:
#Changing Ranks to Categoricals
df['Rank'] = 'Cash Only'

win = ['WIN','1st','1st-4th']
for a in win:
    df['Rank']=np.where(df['RankText'] == a, 'Winners', df['Rank'])
    
second = ['2nd', '7 Wins','8 Wins','6 Wins', '3rd']
for a in second:
    df['Rank']=np.where(df['RankText'] == a, 'Runners Up', df['Rank'])

In [58]:
p = Donut(df['Rank'], title = 'Distribution of Winners')
show(p)
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate,num_teams,country,currency,Team_On_Backup,teams,Rank,rankplace_Cash Only,rankplace_Runners Up,rankplace_Winners,TournLength,PriorWins,SecondWins,Finals,TTtoDate
0,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,None,1058,100.0,136,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000,5.0,ua,EUR,Unaffiliated,Unaffiliated,Winners,0,0,1,6,0,0,0,0
1,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,None,1058,100.0,137,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000,5.0,ua,EUR,Unaffiliated,Unaffiliated,Winners,0,0,1,6,1,0,1,1
2,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,None,1058,200.0,257,WIN,1,mTw,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000,5.0,ua,EUR,mTw,Other,Winners,0,0,1,3,2,0,2,2
3,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,None,1058,1400.0,1400,3rd,1,mTw,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667,5.0,ua,USD,mTw,Other,Runners Up,0,1,0,6,3,0,3,3
4,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,None,1058,100.0,137,WIN,1,mTw,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000,5.0,ua,EUR,mTw,Other,Winners,0,0,1,6,3,1,4,4
5,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,None,1058,100.0,139,WIN,1,mTw,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000,5.0,ua,EUR,mTw,Other,Winners,0,0,1,6,4,1,5,5
6,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,None,1058,3000.0,3000,4th,1,mTw,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667,5.0,ua,USD,mTw,Other,Cash Only,1,0,0,4,5,1,6,6
7,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,None,1058,200.0,274,WIN,1,mTw,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286,5.0,ua,EUR,mTw,Other,Winners,0,0,1,3,5,1,6,7
8,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,None,1058,200.0,267,WIN,1,mTw,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000,5.0,ua,EUR,mTw,Other,Winners,0,0,1,3,6,1,7,8
9,24.0,ua,RUB,DIMAGA,1986-06-03,2010-11-28,0.031859,151,Dmytro,Filipchuk,None,1058,20000.0,637,1st,1,mTw,66288.11,ASUS Autumn 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-28,6384,5747,9,638.555556,5.0,ua,Other,mTw,Other,Winners,0,0,1,5,7,1,8,9


In [39]:
#from Feature_Engineering
#prize pools
tourney_totals = df.groupby('TournamentName')['Prize_USD'].sum()
description = list(tourney_totals.index)
tourney_totals = list(tourney_totals)
df_mrg = pd.DataFrame({'TournamentName':description, 'PrizePool':tourney_totals})
df.merge(df_mrg, on=['TournamentName'],how='right')

#rankplace dummies
df['rankplace']=df['Rank']
df = pd.get_dummies(data=df, columns = ['rankplace'])

In [40]:
def tlength(tournament):
    return len(tournament.split())

df['TournLength']=df['TournamentName'].apply(tlength)

#Wins iterator
dfid = df[df['PlayerId']==1058]
dfid.index = dfid['date']
dfid = dfid.sort_index()
dfid['PriorWins']=dfid.rankplace_Winners.cumsum()-dfid.rankplace_Winners
dfid['SecondWins']=dfid['rankplace_Runners Up'].cumsum()-dfid['rankplace_Runners Up']
dfid['Finals']=dfid['PriorWins']+dfid['SecondWins']
dfid['TTtoDate']=np.arange(len(dfid))
pid = df['PlayerId'].unique()
for i in pid:
    if i != 1058:
        dfi = df[df['PlayerId']==i]
        dfi.index = dfi['date']
        dfi = dfi.sort_index()
        dfi['PriorWins']=dfi.rankplace_Winners.cumsum()-dfi.rankplace_Winners
        dfi['SecondWins']=dfi['rankplace_Runners Up'].cumsum()-dfi['rankplace_Runners Up']
        dfi['Finals']=dfi['PriorWins']+dfi['SecondWins']
        dfi['TTtoDate']=np.arange(len(dfi))
        dfid = dfid.append(dfi)
df = dfid

In [3]:
# df = df.set_index(np.arange(len(df)))
# df_messy = df
# df_messy.to_json('messy.json')
# df_clean = df
df_clean = pd.read_json('messy.json')

In [4]:
df_clean

,Age_at_win,AverageWinsToDate,CountryCode,CumulativePrizeToDate,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,Finals,...,country,currency,date,num_teams,rankplace_Cash Only,rankplace_Runners Up,rankplace_Winners,team_history,teams,totalsc2
0,23.0,0.000000,ua,136,EUR,DIMAGA,5.181408e+11,1267315200000,1.361800,0,...,ua,EUR,2010-02-28,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Unaffiliated,85
1,23.0,136.000000,ua,273,EUR,DIMAGA,5.181408e+11,1268524800000,1.376900,1,...,ua,EUR,2010-03-14,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Unaffiliated,85
10,24.0,638.400000,ua,6651,EUR,DIMAGA,5.181408e+11,1295740800000,1.336525,9,...,ua,EUR,2011-01-23,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Other,85
100,20.0,11777.400000,kr,201661,USD,MC,6.771168e+11,1310256000000,1.000000,9,...,kr,USD,2011-07-10,5,0,1,0,"[{'team': 'CJ Entus', 'history': [146396160000...",Old Generations,108
1000,17.0,816.250000,kr,11888,KRW,RagnaroK,8.417088e+11,1385164800000,0.000942,4,...,kr,KRW,2013-11-23,4,0,1,0,"[{'team': 'CJ Entus', 'history': [142188480000...",Other,38
10000,NaN,0.000000,cn,244,USD,Blue,NaN,1452384000000,1.000000,0,...,cn,USD,2016-01-10,0,0,1,0,[],Unaffiliated,1
10001,NaN,0.000000,no,182,NOK,Adrian,NaN,1428192000000,0.121850,0,...,Other,Other,2015-04-05,0,0,0,1,None,Unaffiliated,2
10002,NaN,182.000000,no,240,NOK,Adrian,NaN,1458950400000,0.117728,1,...,Other,Other,2016-03-26,0,1,0,0,None,Unaffiliated,2
10003,NaN,0.000000,ca,150,USD,Imagine,NaN,1342310400000,1.000000,0,...,Other,USD,2012-07-15,1,1,0,0,"[{'team': 'FlipSid3 Tactics', 'history': [1404...",Unaffiliated,3
10004,NaN,150.000000,ca,226,USD,Imagine,NaN,1430611200000,1.000000,0,...,Other,USD,2015-05-03,1,1,0,0,"[{'team': 'FlipSid3 Tactics', 'history': [1404...",Other,3


In [5]:
to_drop = [ 'rankplace_Cash Only', 'rankplace_Runners Up', 'rankplace_Winners',
            'CountryCode','CurrencyCode','CurrentHandle','ExchangeRate','GameId','NameFirst','NameLast','Note',
            'PlayerId','Prize','RankText','TeamPlayers','Team_On','TournamentName','team_history',
            'Team_On_Backup','DOB','EndDate','TotalUSDPrize', 'Prize_USD',
            'num_teams','date','totalsc2',
            'CumulativePrizeToDate', 'TournLength','AverageWinsToDate','SecondWins','PriorWins','TTtoDate']
test_here = ['Finals',]
df_dropped = df_clean.drop(to_drop, axis=1)

In [6]:
df_dropped.head()

,Age_at_win,Finals,PrizeToDate,Rank,WinsToDate,country,currency,teams
0,23.0,0,0,Winners,0,ua,EUR,Unaffiliated
1,23.0,1,136,Winners,1,ua,EUR,Unaffiliated
10,24.0,9,6384,Winners,10,ua,EUR,Other
100,20.0,9,176661,Runners Up,15,kr,USD,Old Generations
1000,17.0,4,9795,Runners Up,12,kr,KRW,Other


In [7]:
df_dums = pd.get_dummies(data=df_dropped, columns = ['Rank','currency','teams','country'])
df_dums['Age_at_win'] = df_dums['Age_at_win'].fillna(df_dums['Age_at_win'].mean())

In [8]:
df_dums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19969 entries, 0 to 9999
Data columns (total 50 columns):
Age_at_win                             19969 non-null float64
Finals                                 19969 non-null int64
PrizeToDate                            19969 non-null int64
WinsToDate                             19969 non-null int64
Rank_Cash Only                         19969 non-null uint8
Rank_Runners Up                        19969 non-null uint8
Rank_Winners                           19969 non-null uint8
currency_AUD                           19969 non-null uint8
currency_CNY                           19969 non-null uint8
currency_EUR                           19969 non-null uint8
currency_KRW                           19969 non-null uint8
currency_Other                         19969 non-null uint8
currency_USD                           19969 non-null uint8
teams_CJ Entus                         19969 non-null uint8
teams_Dead Pixels                      19969 non-n

In [9]:
#prep for classification
df_dums['y']= df_dums['Rank_Runners Up'] + df_dums['Rank_Winners']
df_class = df_dums.drop(['Rank_Runners Up','Rank_Cash Only','Rank_Winners'], axis=1)
#test to see if there is leakage in the average/total:
y = df_class.pop('y')
df_class.head()

,Age_at_win,Finals,PrizeToDate,WinsToDate,currency_AUD,currency_CNY,currency_EUR,currency_KRW,currency_Other,currency_USD,...,country_Other,country_cn,country_de,country_fr,country_kr,country_pl,country_ru,country_se,country_ua,country_us
0,23.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,23.0,1,136,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10,24.0,9,6384,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
100,20.0,9,176661,15,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1000,17.0,4,9795,12,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
X, X_finaltest, y, y_final_test = train_test_split(df_class,y, random_state=314)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
def standard_confusion_matrix(y_true, y_pred):
    """Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_true, y_pred)
    return np.array([[tp, fp], [fn, tn]])

def roc_curve(probabilities, labels):
    '''
    INPUT: numpy array, numpy array
    OUTPUT: list, list, list

    Take a numpy array of the predicted probabilities and a numpy array of the
    true labels.
    Return the True Positive Rates, False Positive Rates and Thresholds for the
    ROC curve.
    '''
    
    thresholds = np.sort(probabilities)

    tprs = []
    fprs = []

    num_positive_cases = sum(labels)
    num_negative_cases = len(labels) - num_positive_cases

    for threshold in thresholds:
        # With this threshold, give the prediction of each instance
        predicted_positive = probabilities >= threshold
        # Calculate the number of correctly predicted positive cases
        true_positives = np.sum(predicted_positive * labels)
        # Calculate the number of incorrectly predicted positive cases
        false_positives = np.sum(predicted_positive) - true_positives
        # Calculate the True Positive Rate
        tpr = true_positives / float(num_positive_cases)
        # Calculate the False Positive Rate
        fpr = false_positives / float(num_negative_cases)

        fprs.append(fpr)
        tprs.append(tpr)

    return tprs, fprs, thresholds.tolist()

In [25]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

probabilities = clf.predict_proba(X_test)[:, 1]
tprLR, fprLR, thresholds = roc_curve(probabilities, y_test)

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Train Crossval Accuracy: [ 0.63967972  0.65360641  0.63713268  0.650935    0.64203028]
Predicted Accuracy: 0.644676815723
Test Accuracy: 0.652243589744
[[1245  711]
 [ 591 1197]]


Baseline:
Train Crossval Accuracy: [ 0.72051624  0.71918113  0.71428571  0.71460374  0.72071269]
Predicted Accuracy: 0.717859904684
Test Accuracy: 0.717414529915
[[  14   18]
 [1040 2672]]

+ Average
Train Crossval Accuracy: [ 0.72051624  0.72051624  0.71428571  0.71593945  0.722049  ]
Predicted Accuracy: 0.718661329545
Test Accuracy: 0.717681623932
[[  10   13]
 [1044 2677]]
 
 Baseline (1+2):
 Train Crossval Accuracy: [ 0.64441478  0.64307966  0.65583259  0.64870882  0.64692787]
Predicted Accuracy: 0.647792743149
Test Accuracy: 0.65438034188
[[1222  638]
 [ 656 1228]]

In [26]:
clf = GradientBoostingClassifier(learning_rate=.01, max_features=10,n_estimators=1000)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

probabilities = clf.predict_proba(X_test)[:, 1]
tprGB, fprGB, thresholds = roc_curve(probabilities, y_test)

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Train Crossval Accuracy: [ 0.64724199  0.67230632  0.65316118  0.65182547  0.63802315]
Predicted Accuracy: 0.652511622085
Test Accuracy: 0.670138888889
[[1290  689]
 [ 546 1219]]


Baseline Test (No Engineered Features):
Train Crossval Accuracy: [ 0.73965287  0.73742768  0.7329773   0.73063224  0.73853007]
Predicted Accuracy: 0.735844031363
Test Accuracy: 0.729166666667
[[ 130   90]
 [ 924 2600]]

Train Crossval Accuracy: [ 0.73920783  0.73742768  0.73164219  0.730187    0.7403118 ]
Predicted Accuracy: 0.735755301345
Test Accuracy: 0.729433760684
[[ 127   86]
 [ 927 2604]]

Test with AverageToDate:
Train Crossval Accuracy: [ 0.73787272  0.73876279  0.73119715  0.73152271  0.74253898]
Predicted Accuracy: 0.736378869662
Test Accuracy: 0.732905982906
[[ 143   89]
 [ 911 2601]]

In [15]:
chnce = 1 - (5574+4406)/(5574+4406+9989)
print('Random Chance: ' + str(chnce))

Random Chance: 0.5002253492914017


In [45]:
clf = RandomForestClassifier(max_depth=4, n_jobs=1000, max_features=30)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

probabilities = clf.predict_proba(X_test)[:, 1]
tprRF, fprRF, thresholds = roc_curve(probabilities, y_test)

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Train Crossval Accuracy: [ 0.62188612  0.65271594  0.64069457  0.63668744  0.63089938]
Predicted Accuracy: 0.636576689916
Test Accuracy: 0.652243589744
[[1354  820]
 [ 482 1088]]



Train Crossval Accuracy: [ 0.74365821  0.73876279  0.73075211  0.73196794  0.72873051]
Predicted Accuracy: 0.734774314995
Test Accuracy: 0.732638888889
[[ 146   93]
 [ 908 2597]]

In [28]:
probabilities = clf.predict_proba(X_test)[:, 1]
tprRF, fprLRF, thresholds = roc_curve(probabilities, y_test)

In [29]:
def create_line(data, title):
    source = ColumnDataSource(data)
    plot = figure(title = title)
    plot.line('date','Age_at_win',source=source)
    return plot

In [30]:
plot = create_line(df_clean, 'age')

In [7]:
show(plot)

In [51]:
TOOLS = [HoverTool(tooltips=[('FPR:','@x'),('TPR','@y')])]
pal = Spectral11
p = figure(x_axis_label='False Positive Rate', y_axis_label='True Positive Rate', tools=TOOLS)
xvals = np.arange(0,1.01,.01)
yvals = np.arange(0,1.01,.01)

p.line(fprRF, tprRF, legend = 'Logistic Regression', line_width=3, line_color=pal[0])
p.line(fprGB, tprGB, legend = 'Gradient Boost', line_width=3, line_color=pal[1])
p.line(fprLR, tprLR, legend = 'Random Forest', line_width=3, line_color=pal[2])
p.line(xvals,yvals, legend = 'Random Chance', line_width=3, line_color=pal[3])
p.legend.location = 'bottom_right'
show(p)

In [53]:
from bokeh.models import HoverTool
from bokeh.palettes import Spectral11
#Additionally Viridis11, Inferno11 good.
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

output_notebook()

TOOLS = [HoverTool(tooltips=[('FPR:','@x'),('TPR','@y')])]
pal = Spectral11
p = figure(x_axis_label='False Positive Rate', y_axis_label='True Positive Rate', tools=TOOLS)
xvals = np.arange(0,1.01,.01)
yvals = np.arange(0,1.01,.01)

p.line(fprRF, tprRF, legend = 'Logistic Regression', line_width=3, line_color=pal[0])
p.line(fprGB, tprGB, legend = 'Gradient Boost', line_width=3, line_color=pal[1])
p.line(fprLR, tprLR, legend = 'Random Forest', line_width=3, line_color=pal[2])
p.line(xvals,yvals, legend = 'Random Chance', line_width=3, line_color=pal[3])
p.legend.location = 'bottom_right'
show(p)

Loading BokehJS ...

In [52]:
def roc_curve(probabilities, labels):
    '''
    INPUT: numpy array, numpy array
    OUTPUT: list, list, list

    Take a numpy array of the predicted probabilities and a numpy array of the
    true labels.
    Return the True Positive Rates, False Positive Rates and Thresholds for the
    ROC curve.
    '''
    
    thresholds = np.sort(probabilities)

    tprs = []
    fprs = []

    num_positive_cases = sum(labels)
    num_negative_cases = len(labels) - num_positive_cases

    for threshold in thresholds:
        # With this threshold, give the prediction of each instance
        predicted_positive = probabilities >= threshold
        # Calculate the number of correctly predicted positive cases
        true_positives = np.sum(predicted_positive * labels)
        # Calculate the number of incorrectly predicted positive cases
        false_positives = np.sum(predicted_positive) - true_positives
        # Calculate the True Positive Rate
        tpr = true_positives / float(num_positive_cases)
        # Calculate the False Positive Rate
        fpr = false_positives / float(num_negative_cases)

        fprs.append(fpr)
        tprs.append(tpr)

    return tprs, fprs, thresholds.tolist()